In [2]:

class MysqlConn(object):
    def __init__(self, config_json_str):
        # self.mysql_config = {"host": "10.22.128.150",  "port": 3317, "db": "bigdata", "user": "talend_load",
        #   "passwd": "s9t5gNThn2vqWM7c" , "charset" : "utf8"}
        # self.mysql_config = {"host": "db-bigdata.wmcloud-qa.com",  "port": 3312, "db": "bigdata", "user": "app_bigdata_ro",
        #   "passwd": "Welcome_20141217"}
        self.mysql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mysql_conn = pymysql.connect(**self.mysql_config)
        return mysql_conn

class MssqlConn(object):
    def __init__(self, config_json_str):
        # self.mssql_config = {"server": "sh-datayesdb.wmcloud-dev.com",  "port": 1433, "database": "datayesdb",
        #     "user": "talend_load", "password": "Welcome01"}
        self.mssql_config = json.loads(config_json_str, encoding='utf-8')

    def connect(self):
        mssql_conn = pymssql.connect(**self.mssql_config)
        return mssql_conn

    def __str__(self):
        return json.dumps(self.mssql_config, ensure_ascii=False, encoding='utf-8')



In [8]:
import json
import pymysql
import pymssql
# MySQL
# bigdata={"host":"security03-dev.datayes.com","port":3306,"user":"talend_load","passwd":"NCph1G9BQT3DuQj","db":"bigdata","charset":"utf8"}
# dateyesdbp={"host":"db-datayesdb-ro.wmcloud.com","port":3313,"user":"app_dataqa_ro","passwd":"Welcome20140820","db":"datayesdbp","charset":"utf8"}

# # dateyesdbp={"host":"db-datayesdb.wmcloud.com","port":3312,"user":"app_marketdata","passwd":"JDFJ8dfasd8aKfu","db":"datayesdbp","charset":"utf8"}

# SQLserverdb-mssql02.datayes-stg.com
# health_40={"server":"db-mssql02.datayes-stg.com","port":1433,"database":"health","user":"talend_load","password":"s9t5gNThn2vqWM7c","charset":"utf8"}
health_db={"server":"db-mssql02.datayes-stg.com","port":1433,"database":"health","user":"datateam_ro","password":"rodatateam123","charset":"utf8"}
health_db = {"server":"sh-dm-db03.datayes.com","port":1433,"database":"health","user":"talend_load","password":"Welcome2046","charset":"utf8"}

# bigdata_conn = MysqlConn(json.dumps(bigdata)).connect()
# dateyesdbp_conn = MysqlConn(json.dumps(dateyesdbp)).connect()

# health_conn = MssqlConn(json.dumps(health_40)).connect()
health_conn = MssqlConn(json.dumps(health_db)).connect()


In [28]:
import pandas as pd

In [4]:
# from lib.config import Config, logger
# from utils import data_api_prefix
# from pandas import DataFrame

from bson import json_util as json
import requests
import types

class APIVisitor(object):
    SECID_SIZE = 200  # 接收数组参数的最大长度

    def __init__(self, token, url, username=""):
        self.token = token
        self.url = url
        self.username = username
        # self.cfg = Config()

    def toStr(self, val):
        if val is None:
            val = ""
        elif isinstance(val, types.ListType):
            # elif type(val) is types.ListType:
            val = ",".join(val)
        elif isinstance(val, types.IntType):
            # elif type(val) is types.IntType:
            val = str(val)
        return val.decode("utf8")

    def getcontent(self, params):
        """
        
        :param params: piMedicinalname=&piBigsortid=&piBigsortname=&piMedicineparentid=&piMedicineparentname=&piS
        :return:
        """
        paramStr = ["%s=%s" % (key, self.toStr(params[key])) for key in params.keys()]
        info = "&".join(paramStr)
        print info
        api = self.url + "?" + info  # "/api/%s/%s.json?%s" % (api_type, api_name, searches)
        # logger.info(u"get data with api %s" % self.url)
        resp = requests.get(url=api, headers={"username": self.username,
                                                   "token": self.token}, verify=False)
        code, ret_json = resp.status_code, resp.json()
        if code == 200 and ret_json['retCode'] == 1:
            return ret_json['data']
        # else:
        #     logger.info(u"failed with http status code %s and response content %s, url %s"
        #                 % (code, json.dumps(ret_json, ensure_ascii=False), api))
        return None


# getCiDrugsFilter

In [5]:
import calendar

# 数据的日期为离散值， 为每个季度末。
# report_month = [3,6,9,12]
report_month = ['03', '06', '09', '12']

datelist = []
for year in range(2012, 2020):
    for mon in report_month:
        res= calendar.monthrange(year,int(mon))
        datelist.append("%s-%s-%s" % (str(year),str(mon),str(res[1])))
# datelist

In [6]:
# getCiDrugsFilter 
username = "rrp"
token = "37468D37D9CE3B7B36749FACF712C245"
getCiDrugsFilter = "https://124.172.190.114:8443/api/health/getCiDrugsFilter.json?"

In [7]:
# API类用于获取数据
ci_getCiDrugsFilter = APIVisitor(token, getCiDrugsFilter, username)
# getCiDrugsFilter = ci_getCiDrugsFilter.getcontent(params)


In [15]:
# 整合成一个函数 ， 

def getCiDrugssalesFiler(params):
#     getCiDrugssales = "https://124.172.190.114:8443/api/health/getCiDrugssales.json"
    ci_drugssales = APIVisitor(token, ci_getCiDrugsFilter, username)
    data = ci_drugssales.getcontent(params)
    
    # 将API返回的数据转换成df
    data_df = pd.DataFrame(data)
#     for i,item in enumerate(data_drugssales):
#         df = pd.DataFrame(item, index=[i])
#         df_drugssales = df_drugssales.append(df)
#     data_df.info()
    
    #     df_drugssales_merge.head()
    data_df.columns=[u'DATE',u'FREQUENCY',u'PMD_CITY',u'PMD_CLASS', u'PMD_COMPANYID',u'PMD_COMPANYNAME',u'PMD_DOSFORM',u'PMD_MEDICINALID',u'PMD_MEDICINALINFORMATIONID',u'PMD_MEDICINALNAME',u'PMD_REMARK1',u'PMD_SPECIFICIAL', u'PMD_USEMETHOD']
    #     df_drugssales_merge.head()
    return data_df

In [16]:
# ID
# DATE
# FREQUENCY
# PMD_CITY
# PMD_MEDICINALINFORMATIONID
# PMD_MEDICINALID
# PMD_MEDICINALNAME
# PMD_COMPANYID
# PMD_COMPANYNAME
# PMD_DOSFORM
# PMD_SPECIFICIAL
# PMD_USEMETHOD
# PMD_REMARK1
# PMD_CLASS
# PMD_REMARK2


In [17]:
# slaes数据需要跟另外两张表中的部分数据进行合并，等到所需字段
# df_DrugsFilter.columns = df_drugsclassification_columns
# df_drugs.columns = df_drugs_columns

target_table = "ci_drugs_filter"
uniq_keys = ['DATE', 'PMD_CITY', 'PMD_MEDICINALINFORMATIONID', 'PMD_COMPANYID']
# DATE, PMD_CITY, PMD_MEDICINALINFORMATIONID, PMD_COMPANYID
# PMD_YEAR, PMD_QUARTER, PMD_CITY, PMD_MEDICINALINFORMATIONID, PMD_COMPANYID,PMD_DOSFORM
# target_conn = loader.create_connection(conn_type='ci_drugssales')


for d in datelist[7:8]:
    where_condition="where date=%s" % (d)
#     d='2015-12-31'
    params = {
#         "beginDate":d,
#         "endDate":d,
        "date":d
    }
    
    data_df = getCiDrugssalesFiler(params)
    
#     base.merge_2_table(
#         data_df,
#         target_table,
#         uniq_keys,
#         where_condition,
#         target_conn)
# data_df.columns=[ u'date',u'PMD_COMPANYID',u'PMD_COMPANYNAME',u'PMD_MEDICINALID',u'PMD_MEDICINALINFORMATIONID',u'PMD_MEDICINALSALEMONEY',u'PMD_MEDICINALSALENUMBER',u'PMD_YEAR', u'PMD_QUARTER',u'PMD_REMARK1',u'PMD_DOSFORM',u'PMD_SPECIFICIAL',u'PMD_CLASS']


date=2013-12-31


TypeError: unsupported operand type(s) for +: 'APIVisitor' and 'str'

In [ ]:
# ID
# DATE
# FREQUENCY
# PMD_CITY
# PMD_MEDICINALINFORMATIONID
# PMD_MEDICINALID
# PMD_MEDICINALNAME
# PMD_COMPANYID
# PMD_COMPANYNAME
# PMD_DOSFORM
# PMD_SPECIFICIAL
# PMD_USEMETHOD
# PMD_REMARK1
# PMD_CLASS
# PMD_REMARK2
data_df.columns=[u'DATE',u'FREQUENCY',u'PMD_CITY',u'PMD_CLASS', u'PMD_COMPANYID',u'PMD_COMPANYNAME',u'PMD_DOSFORM',u'PMD_MEDICINALID',u'PMD_MEDICINALINFORMATIONID',u'PMD_MEDICINALNAME',u'PMD_REMARK1',u'PMD_SPECIFICIAL', u'PMD_USEMETHOD']

In [20]:
data_df.head()

,date,frequency,pmdCity,pmdClass,pmdCompanyid,pmdCompanyname,pmdDosform,pmdMedicinalid,pmdMedicinalinformationid,pmdMedicinalname,pmdRemark1,pmdSpecificial,pmdUsemethod
0,2013-03-31,Q,重庆,2,M1250,贵州汉方药业,散剂、颗粒剂,ZB0200-D003194,D0031940100CAA0322T010,黄芪,10,15g,内服
1,2013-03-31,Q,重庆,2,M4393,新疆维吾尔药业,散剂、颗粒剂,ZV0100-D002234,D0022340100CAA0237T006,复方木尼孜其,6,12g,内服
2,2013-03-31,Q,重庆,2,M2160,吉林集安益盛药业,注射剂,ZA0100-D007437,D0074370200LCB0425T001,消痔灵,1,10ml:400mg,注射
3,2013-03-31,Q,重庆,2,M4870,重庆巨琪诺美制药,片剂,ZN0200-D005657,D0056570200AAA1332T024,全天麻,24,600mg,内服
4,2013-03-31,Q,重庆,2,M3857,四川升和药业,注射剂,ZC0300-D001103,D0011030700LCB0359T001,丹参,1,10ml,注射


In [19]:
data_df["pmdCity"] = data_df["pmdCity"].apply(lambda x: x[:-1] if x.endswith(u'市') else x)



In [30]:
data_df = None
print ("%s" % data_df.shape[0])
data_df

AttributeError: 'NoneType' object has no attribute 'shape'

In [26]:
# key:PI＿ID
params = {
    "FIELD":"",
    "piID":"",
    "piMedicinalname":"",
    "piBigsortid":""
}
ci_getCiDrugsFilter = APIVisitor(token, getCiDrugsFilter, username)
getCiDrugsFilter = ci_getCiDrugsFilter.getcontent(params)


FIELD=&piBigsortid=&piID=&piMedicinalname=


c:\python27\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
# authetification 
username = "rrp"
token = "37468D37D9CE3B7B36749FACF712C245"

In [26]:
# getCiDrugsclassification = "https://124.172.190.114:8443/api/health/getCiDrugsclassification.json?FIELD=&piID=&piMedicinalname=&piBigsortid=&piBigsortname=&piMedicineparentid=&piMedicineparentname=&piS"
getCiDrugsclassification = "https://124.172.190.114:8443/api/health/getCiDrugsclassification.json"
# key:PI＿ID
params = {
    "FIELD":"",
    "piID":"",
    "piMedicinalname":"",
    "piBigsortid":""
}
ci_drugsclassification = APIVisitor(token, getCiDrugsclassification, username)
drugsclassification = ci_drugsclassification.getcontent(params)


FIELD=&piBigsortid=&piID=&piMedicinalname=


c:\python27\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [77]:
# pd.DataFrame(drugsclassification)

In [7]:
drugsclassification[0]

{u'piBigsortid': u'ZC',
 u'piBigsortname': u'\u5fc3\u8111\u8840\u7ba1\u75be\u75c5\u7528\u836f',
 u'piID': u'ZC0200-D001964',
 u'piMedicinalatccode': u'ZC0200',
 u'piMedicinalname': u'\u590d\u65b9\u4e39\u86ed',
 u'piMedicineparentid': u'ZC02',
 u'piMedicineparentname': u'\u8111\u8840\u7ba1\u75be\u75c5\u7528\u836f',
 u'pmdClass': 2}

In [78]:
# # 将API返回的数据转换成df
import pandas as pd
# df_drugsclassification = pd.DataFrame()
# for i,item in enumerate(drugsclassification):
#     df = pd.DataFrame(item, index=[i])
#     df_drugsclassification = df_drugsclassification.append(df)

In [71]:
df_drugsclassification.head()
# df_drugsclassification.info()


,piBigsortid,piBigsortname,piID,piMedicinalatccode,piMedicinalname,piMedicineparentid,piMedicineparentname,piSmallsortid,piSmallsortname,pmdClass
0,ZC,心脑血管疾病用药,ZC0200-D001964,ZC0200,复方丹蛭,ZC02,脑血管疾病用药,NaN,NaN,2
1,ZC,心脑血管疾病用药,ZC0300-D001971,ZC0300,复方党参,ZC03,心血管疾病用药,NaN,NaN,2
2,ZC,心脑血管疾病用药,ZC0200-D001978,ZC0200,复方地龙,ZC02,脑血管疾病用药,NaN,NaN,2
3,ZK,五官科用药,ZK0100-D002000,ZK0100,复方冬凌草,ZK01,咽喉用药,NaN,NaN,2
4,ZC,心脑血管疾病用药,ZC0300-D002003,ZC0300,复方杜仲,ZC03,心血管疾病用药,NaN,NaN,2


In [68]:
df_drugsclassification_columns = df_drugsclassification.columns

In [70]:
# df_drugsclassification.columns= df_drugsclassification_columns

In [10]:
# import pandas as pd

# to get party_id list
sql = """
select *
from ci_drugsclassification 
"""
data = pd.read_sql(sql, health_conn)
data.columns

Index([u'ID', u'PI_ID', u'PI_MEDICINALNAME', u'PI_MEDICINALATCCODE',
       u'PI_MEDICINEPARENTNAME', u'PI_MEDICINEPARENTID', u'PI_BIGSORTNAME',
       u'PI_BIGSORTID', u'PI_REMARK1', u'PI_REMARK2', u'PI_REMARK3',
       u'CREATE_DATE', u'CREATE_USER', u'UPDATE_DATE', u'UPDATE_USER',
       u'VERSION', u'STATUS', u'PMD_CLASS', u'PI_SMALLSORTNAME',
       u'PI_SMALLSORTID', u'PMD_DATABASE', u'tempid', u'ETL_CRC',
       u'QA_RULE_CHK_FLG', u'QA_MANUAL_FLG', u'QA_ACTIVE_FLG', u'CREATE_BY',
       u'CREATE_TIME', u'UPDATE_BY', u'UPDATE_TIME', u'TMSTAMP'],
      dtype='object')

In [69]:
df_drugsclassification.columns=[u'PI_BIGSORTID',u'PI_BIGSORTNAME', u'PI_ID', u'PI_MEDICINALATCCODE',u'PI_MEDICINALNAME', u'PI_MEDICINEPARENTID',u'PI_MEDICINEPARENTNAME',
       u'PI_SMALLSORTID',u'PI_SMALLSORTNAME',u'PMD_CLASS']
df_drugsclassification.head()

,PI_BIGSORTID,PI_BIGSORTNAME,PI_ID,PI_MEDICINALATCCODE,PI_MEDICINALNAME,PI_MEDICINEPARENTID,PI_MEDICINEPARENTNAME,PI_SMALLSORTID,PI_SMALLSORTNAME,PMD_CLASS
0,ZC,心脑血管疾病用药,ZC0200-D001964,ZC0200,复方丹蛭,ZC02,脑血管疾病用药,NaN,NaN,2
1,ZC,心脑血管疾病用药,ZC0300-D001971,ZC0300,复方党参,ZC03,心血管疾病用药,NaN,NaN,2
2,ZC,心脑血管疾病用药,ZC0200-D001978,ZC0200,复方地龙,ZC02,脑血管疾病用药,NaN,NaN,2
3,ZK,五官科用药,ZK0100-D002000,ZK0100,复方冬凌草,ZK01,咽喉用药,NaN,NaN,2
4,ZC,心脑血管疾病用药,ZC0300-D002003,ZC0300,复方杜仲,ZC03,心血管疾病用药,NaN,NaN,2


In [ ]:

# base = DB_Base()
# base.merge_2_table(
#     result_df,
#     target_table,
#     uniq_keys,
#     where_condition,
#     target_conn)

In [24]:
"https://124.172.190.114:8443/api/health/getCiDrugs.json?FIELD=&mdiID=&mdiAtccode=&piID=&mdiSfdaapprovalid=&mdiMedicinename=&mdiUsemethod=&mdiDosform=&mdiSpecificial="
getCiDrugs = "https://124.172.190.114:8443/api/health/getCiDrugs.json"
# key MDI_ID
# params = {
#     "FIELD":"",
#     "piID":"",
#     "piMedicinalname":"",
#     "piBigsortid":""
# }
ci_drugs = APIVisitor(token, getCiDrugs, username)
data_drugs = ci_drugs.getcontent({})
# data_drugs

c:\python27\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [25]:
# 将API返回的数据转换成df
df_drugs = pd.DataFrame()
for i,item in enumerate(data_drugs):
    df = pd.DataFrame(item, index=[i])
    df_drugs = df_drugs.append(df)
df_drugs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61113 entries, 0 to 61112
Data columns (total 9 columns):
mdiAtccode           61113 non-null object
mdiDosform           61113 non-null object
mdiID                61113 non-null object
mdiMedicinename      61113 non-null object
mdiRemark1           61113 non-null object
mdiSfdaapprovalid    61113 non-null object
mdiSpecificial       61113 non-null object
mdiUsemethod         61113 non-null object
piID                 61113 non-null object
dtypes: object(9)
memory usage: 4.7+ MB


In [19]:
df_drugs.head()


,mdiAtccode,mdiDosform,mdiID,mdiMedicinename,mdiRemark1,mdiSfdaapprovalid,mdiSpecificial,mdiUsemethod,piID
0,ZC0200,片剂,D0019640100AAA0940T056,复方丹蛭,60,复方丹蛭片,360mg,内服,ZC0200-D001964
1,ZC0200,片剂,D0019640100AAA0925T056,复方丹蛭,60,复方丹蛭片,350mg,内服,ZC0200-D001964
2,ZC0300,片剂,D0019710100AAA1156T082,复方党参,100,复方党参片,500mg,内服,ZC0300-D001971
3,ZC0300,片剂,D0019710100AAA1156T046,复方党参,50,复方党参片,500mg,内服,ZC0300-D001971
4,ZC0300,片剂,D0019710100AAA1156T072,复方党参,80,复方党参片,500mg,内服,ZC0300-D001971


In [72]:
df_drugs_columns = df_drugs.columns

In [18]:
# import pandas as pd
sql = """
select * from ci_drugs limit where id<10
"""
data = pd.read_sql(sql, health_conn)
data.columns

Index([u'ID', u'MDI_SFDAAPPROVALID', u'MDI_UNIFIEDID', u'MDI_ATCCODE',
       u'MDI_MEDICINENAME', u'MDI_DOSFORM', u'MDI_SPECIFICIAL',
       u'MDI_USEMETHOD', u'MDI_ID', u'PI_ID', u'MDI_REMARK1', u'MDI_REMARK2',
       u'MDI_REMARK3', u'MDI_REMARK4', u'MDI_REMARK5', u'CREATE_DATE',
       u'CREATE_USER', u'UPDATE_DATE', u'UPDATE_USER', u'VERSION', u'STATUS',
       u'ETL_CRC', u'QA_RULE_CHK_FLG', u'QA_MANUAL_FLG', u'QA_ACTIVE_FLG',
       u'CREATE_BY', u'CREATE_TIME', u'UPDATE_BY', u'UPDATE_TIME', u'TMSTAMP'],
      dtype='object')

In [21]:
df_drugs.columns=[u'MDI_ATCCODE',u'MDI_DOSFORM',u'MDI_ID', u'MDI_MEDICINENAME',u'MDI_REMARK1',u'MDI_SFDAAPPROVALID',u'MDI_SPECIFICIAL',u'MDI_USEMETHOD',u'PI_ID']
df_drugs.head()

,MDI_ATCCODE,MDI_DOSFORM,MDI_ID,MDI_MEDICINENAME,MDI_REMARK1,MDI_SFDAAPPROVALID,MDI_SPECIFICIAL,MDI_USEMETHOD,PI_ID
0,ZC0200,片剂,D0019640100AAA0940T056,复方丹蛭,60,复方丹蛭片,360mg,内服,ZC0200-D001964
1,ZC0200,片剂,D0019640100AAA0925T056,复方丹蛭,60,复方丹蛭片,350mg,内服,ZC0200-D001964
2,ZC0300,片剂,D0019710100AAA1156T082,复方党参,100,复方党参片,500mg,内服,ZC0300-D001971
3,ZC0300,片剂,D0019710100AAA1156T046,复方党参,50,复方党参片,500mg,内服,ZC0300-D001971
4,ZC0300,片剂,D0019710100AAA1156T072,复方党参,80,复方党参片,500mg,内服,ZC0300-D001971


In [ ]:
key_drugs = "MDI_ID"

In [ ]:
# 整合成一个函数 ， 

def getCiDrugssales(params):
    getCiDrugssales = "https://124.172.190.114:8443/api/health/getCiDrugssales.json"
    ci_drugssales = APIVisitor(token, getCiDrugssales, username)
    data_drugssales = ci_drugssales.getcontent(params)
    
    # 将API返回的数据转换成df
    df_drugssales = pd.DataFrame()
    for i,item in enumerate(data_drugssales):
        df = pd.DataFrame(item, index=[i])
        df_drugssales = df_drugssales.append(df)
    df_drugssales.info()

    # df_drugssales['date'].astype(date)
    df_drugssales['year'] = df_drugssales['date'].apply(lambda x: x[:])
    df_drugssales['quarter'] = df_drugssales['date'].apply(lambda x: int(x[5:7])/3)
    df_drugssales = df_drugssales.drop(['date','frequency'], axis=1)
    df_drugssales.head()
    

    df_drugssales['mdiID'] = df_drugssales['pmdMedicinalinformationid']
    df_drugssales_merge = df_drugssales.merge(df_drugs[['mdiID','mdiRemark1','mdiDosform','mdiSpecificial']], on='mdiID', how='left') 
    df_drugssales_merge = df_drugssales_merge.drop(["mdiID"], axis=1)

    df_drugssales_merge['piID'] = df_drugssales_merge['pmdMedicinalID']
    df_drugssales_merge = df_drugssales_merge.merge(df_drugsclassification[['piID','pmdClass']], on='piID', how='left') 
    df_drugssales_merge = df_drugssales_merge.drop("piID", axis=1)

    #     df_drugssales_merge.head()
    df_drugssales_merge.columns=[ u'PMD_CITY',u'PMD_COMPANYID',u'PMD_COMPANYNAME',u'PMD_MEDICINALID',u'PMD_MEDICINALINFORMATIONID',u'PMD_MEDICINALSALEMONEY',u'PMD_MEDICINALSALENUMBER',u'PMD_YEAR', u'PMD_QUARTER',u'PMD_REMARK1',u'PMD_DOSFORM',u'PMD_SPECIFICIAL',u'PMD_CLASS']
    #     df_drugssales_merge.head()
    return df_drugssales_merge

In [86]:
import calendar
# 数据的日期为离散值， 为每个季度末。
# report_month = [3,6,9,12]
report_month = ['03', '06', '09', '12']

datelist = []
for year in range(2012, 2020):
    for mon in report_month:
        res= calendar.monthrange(year,int(mon))
        datelist.append("%s-%s-%s" % (str(year),str(mon),str(res[1])))
datelist

['2012-03-31',
 '2012-06-30',
 '2012-09-30',
 '2012-12-31',
 '2013-03-31',
 '2013-06-30',
 '2013-09-30',
 '2013-12-31',
 '2014-03-31',
 '2014-06-30',
 '2014-09-30',
 '2014-12-31',
 '2015-03-31',
 '2015-06-30',
 '2015-09-30',
 '2015-12-31',
 '2016-03-31',
 '2016-06-30',
 '2016-09-30',
 '2016-12-31',
 '2017-03-31',
 '2017-06-30',
 '2017-09-30',
 '2017-12-31',
 '2018-03-31',
 '2018-06-30',
 '2018-09-30',
 '2018-12-31',
 '2019-03-31',
 '2019-06-30',
 '2019-09-30',
 '2019-12-31']

In [ ]:
# slaes数据需要跟另外两张表中的部分数据进行合并，等到所需字段
df_drugsclassification.columns = df_drugsclassification_columns
df_drugs.columns = df_drugs_columns

target_table = "ci_drugssales"
uniq_keys = ['PMD_YEAR', 'PMD_QUARTER', 'PMD_CITY', 'PMD_MEDICINALINFORMATIONID', 'PMD_COMPANYID']
target_conn = loader.create_connection(conn_type='ci_drugssales')


for d in datelist:
    where_condition="where pmd_year=%s and pmd_quarter=%s" % (d[0:4], str(int(d[5:7])/3))
    
    params = {
        "beginDate":d,
        "endDate":d,
    }
    data = getCiDrugssales(params)
    base.merge_2_table(
        data,
        target_table,
        uniq_keys,
        where_condition,
        target_conn)

    


In [ ]:

# base = DB_Base()
# base.merge_2_table(
#     result_df,
#     target_table,
#     uniq_keys,
#     where_condition,
#     target_conn)

In [82]:
"https://124.172.190.114:8443/api/health/getCiDrugssales.json?FIELD=&beginDate=&endDate=&pmdCity=&pmdMedicinalinformationid=&pmdCompanyid=&pmdCompanyname=&pmdMedicinalID="

getCiDrugssales = "https://124.172.190.114:8443/api/health/getCiDrugssales.json"
# PMD_YEAR, PMD_QUARTER, PMD_CITY, PMD_MEDICINALINFORMATIONID, PMD_COMPANYID 
# 关联drugs 表, PMD_MEDICINALINFORMATIONID
params = {
#     "pmdCity":"北京市",
    "beginDate":"2013-03-31",
    "endDate":"2013-03-31"
}
ci_drugssales = APIVisitor(token, getCiDrugssales, username)
data_drugssales = ci_drugssales.getcontent(params)


beginDate=2013-03-31&endDate=2013-03-31


c:\python27\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [83]:
len(data_drugssales)

71686

In [25]:
data_drugssales[0]

{u'date': u'2013-03-31',
 u'frequency': u'Q',
 u'pmdCity': u'\u5317\u4eac\u5e02',
 u'pmdCompanyid': u'M1316',
 u'pmdCompanyname': u'\u8d35\u5dde\u4fe1\u90a6\u5236\u836f',
 u'pmdMedicinalID': u'ZA0600-D003111',
 u'pmdMedicinalinformationid': u'D0031110200AAA0925T082',
 u'pmdMedicinalsalemoney': 928428.3566,
 u'pmdMedicinalsalenumber': 25788.0}

In [96]:
# 将API返回的数据转换成df
df_drugssales = pd.DataFrame(data_drugssales)
# for i,item in enumerate(data_drugssales[:100]):
#     df = pd.DataFrame(item, index=[i])
#     df_drugssales = df_drugssales.append(df)
# df_drugssales.info()



In [94]:
df_drugssales.head()

,pmdCity,pmdCompanyid,pmdCompanyname,pmdMedicinalID,pmdMedicinalinformationid,pmdMedicinalsalemoney,pmdMedicinalsalenumber,year,quarter,mdiID
0,北京市,M1316,贵州信邦制药,ZA0600-D003111,D0031110200AAA0925T082,928428.3566,25788.0,2013,1,D0031110200AAA0925T082
1,北京市,M3809,四川金辉药业,ZF0300-D003587,D0035870500AAA1238T045,67717.9205,3214.0,2013,1,D0035870500AAA1238T045
2,北京市,M2425,江西济民可信药业,ZF0300-D003910,D0039100400AAA1254T056,471059.4568,20910.0,2013,1,D0039100400AAA1254T056
3,北京市,M1847,呼伦贝尔松鹿制药,ZE0300-D007564,D0075640100AAA1156T056,48309.6656,2005.0,2013,1,D0075640100AAA1156T056
4,北京市,M3569,上海医药青岛国风药业,ZC0300-D007902,D0079020100AAA1332T035,298423.1510,9392.0,2013,1,D0079020100AAA1332T035


In [73]:
df_drugssales_columns = df_drugssales.columns

In [97]:
# df_drugssales['date'].astype(date)
df_drugssales['year'] = df_drugssales['date'].apply(lambda x: x[:4])
df_drugssales['quarter'] = df_drugssales['date'].apply(lambda x: int(x[5:7])/3)
df_drugssales = df_drugssales.drop(['date','frequency'], axis=1)
df_drugssales.head()

,pmdCity,pmdCompanyid,pmdCompanyname,pmdMedicinalID,pmdMedicinalinformationid,pmdMedicinalsalemoney,pmdMedicinalsalenumber,year,quarter
0,北京市,M1316,贵州信邦制药,ZA0600-D003111,D0031110200AAA0925T082,928428.3566,25788.0,2013,1
1,北京市,M3809,四川金辉药业,ZF0300-D003587,D0035870500AAA1238T045,67717.9205,3214.0,2013,1
2,北京市,M2425,江西济民可信药业,ZF0300-D003910,D0039100400AAA1254T056,471059.4568,20910.0,2013,1
3,北京市,M1847,呼伦贝尔松鹿制药,ZE0300-D007564,D0075640100AAA1156T056,48309.6656,2005.0,2013,1
4,北京市,M3569,上海医药青岛国风药业,ZC0300-D007902,D0079020100AAA1332T035,298423.1510,9392.0,2013,1


In [51]:
# df_drugs.head()

,mdiAtccode,mdiDosform,mdiID,mdiMedicinename,mdiRemark1,mdiSfdaapprovalid,mdiSpecificial,mdiUsemethod,piID
0,ZC0200,片剂,D0019640100AAA0940T056,复方丹蛭,60,复方丹蛭片,360mg,内服,ZC0200-D001964
1,ZC0200,片剂,D0019640100AAA0925T056,复方丹蛭,60,复方丹蛭片,350mg,内服,ZC0200-D001964
2,ZC0300,片剂,D0019710100AAA1156T082,复方党参,100,复方党参片,500mg,内服,ZC0300-D001971
3,ZC0300,片剂,D0019710100AAA1156T046,复方党参,50,复方党参片,500mg,内服,ZC0300-D001971
4,ZC0300,片剂,D0019710100AAA1156T072,复方党参,80,复方党参片,500mg,内服,ZC0300-D001971


In [98]:
# slaes数据需要跟另外两张表中的部分数据进行合并，等到所需字段
df_drugsclassification.columns = df_drugsclassification_columns
df_drugs.columns = df_drugs_columns

df_drugssales['mdiID'] = df_drugssales['pmdMedicinalinformationid']
df_drugssales_merge = df_drugssales.merge(df_drugs[['mdiID','mdiMedicinename','mdiUsemethod','mdiRemark1','mdiDosform','mdiSpecificial']], on='mdiID', how='left') 
df_drugssales_merge = df_drugssales_merge.drop(["mdiID"], axis=1)

df_drugssales_merge['piID'] = df_drugssales_merge['pmdMedicinalID']
df_drugssales_merge = df_drugssales_merge.merge(df_drugsclassification[['piID','pmdClass']], on='piID', how='left') 
df_drugssales_merge = df_drugssales_merge.drop("piID", axis=1)

df_drugssales_merge.head()

,pmdCity,pmdCompanyid,pmdCompanyname,pmdMedicinalID,pmdMedicinalinformationid,pmdMedicinalsalemoney,pmdMedicinalsalenumber,year,quarter,mdiMedicinename,mdiUsemethod,mdiRemark1,mdiDosform,mdiSpecificial,pmdClass
0,北京市,M1316,贵州信邦制药,ZA0600-D003111,D0031110200AAA0925T082,928428.3566,25788.0,2013,1,护肝宁,内服,100,片剂,350mg,2
1,北京市,M3809,四川金辉药业,ZF0300-D003587,D0035870500AAA1238T045,67717.9205,3214.0,2013,1,金刚藤,内服,48,片剂,520mg,2
2,北京市,M2425,江西济民可信药业,ZF0300-D003910,D0039100400AAA1254T056,471059.4568,20910.0,2013,1,抗宫炎,内服,60,片剂,550mg:500mg,2
3,北京市,M1847,呼伦贝尔松鹿制药,ZE0300-D007564,D0075640100AAA1156T056,48309.6656,2005.0,2013,1,小儿胃宝,内服,60,片剂,500mg,2
4,北京市,M3569,上海医药青岛国风药业,ZC0300-D007902,D0079020100AAA1332T035,298423.1510,9392.0,2013,1,养心氏,内服,36,片剂,600mg,2


In [64]:
# df_drugsclassification.head()
# df_drugssales_merge.info()
# df_drugssales_merge.head()

,PMD_CITY,PMD_COMPANYID,PMD_COMPANYNAME,PMD_MEDICINALID,PMD_MEDICINALINFORMATIONID,PMD_MEDICINALSALEMONEY,PMD_MEDICINALSALENUMBER,PMD_YEAR,PMD_QUARTER,PMD_REMARK1,PMD_DOSFORM,PMD_SPECIFICIAL,PMD_CLASS
0,北京市,M1316,贵州信邦制药,ZA0600-D003111,D0031110200AAA0925T082,928428.3566,25788.0,2013-03-31,1,100,片剂,350mg,2
1,北京市,M3809,四川金辉药业,ZF0300-D003587,D0035870500AAA1238T045,67717.9205,3214.0,2013-03-31,1,48,片剂,520mg,2
2,北京市,M2425,江西济民可信药业,ZF0300-D003910,D0039100400AAA1254T056,471059.4568,20910.0,2013-03-31,1,60,片剂,550mg:500mg,2
3,北京市,M1847,呼伦贝尔松鹿制药,ZE0300-D007564,D0075640100AAA1156T056,48309.6656,2005.0,2013-03-31,1,60,片剂,500mg,2
4,北京市,M3569,上海医药青岛国风药业,ZC0300-D007902,D0079020100AAA1332T035,298423.1510,9392.0,2013-03-31,1,36,片剂,600mg,2


In [14]:
# import pandas as pd

# to get party_id list
sql = """
select * from ci_drugssales where id<10
"""
data = pd.read_sql(sql, health_conn)
data.columns

Index([u'ID', u'PMD_YEAR', u'PMD_QUARTER', u'PMD_CITY',
       u'PMD_MEDICINALSALEMONEY', u'PMD_MEDICINALSALENUMBER',
       u'PMD_MEDICINALNAME', u'PMD_COMPANYNAME', u'PMD_COMPANYID',
       u'PMD_MEDICINALINFORMATIONID', u'PMD_MEDICINALID', u'PMD_ID',
       u'PMD_REMARK1', u'PMD_REMARK2', u'PMD_REMARK3', u'PMD_REMARK4',
       u'PMD_REMARK5', u'CREATE_DATE', u'CREATE_USER', u'UPDATE_DATE',
       u'UPDATE_USER', u'VERSION', u'STATUS', u'PMD_DATABASE', u'PMD_CLASS',
       u'PMD_DOSFORM', u'PMD_SPECIFICIAL', u'PMD_USEMETHOD', u'ETL_CRC',
       u'QA_RULE_CHK_FLG', u'QA_MANUAL_FLG', u'QA_ACTIVE_FLG', u'CREATE_BY',
       u'CREATE_TIME', u'UPDATE_BY', u'UPDATE_TIME', u'TMSTAMP'],
      dtype='object')

### 库里写入了大量重复数据， 需要drop

In [9]:
"""select * from ci_drugssales with (nolock) where pmd_year=2013 and PMD_QUARTER=1 """

# import pandas as pd

# to get party_id list
sql = """
select * from ci_drugssales with (nolock) where pmd_year=2013 and PMD_QUARTER=1
"""
data = pd.read_sql(sql, health_conn)
# data.columns

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216427 entries, 0 to 216426
Data columns (total 37 columns):
ID                            216427 non-null int64
PMD_YEAR                      216427 non-null int64
PMD_QUARTER                   216427 non-null int64
PMD_CITY                      216427 non-null object
PMD_MEDICINALSALEMONEY        216427 non-null float64
PMD_MEDICINALSALENUMBER       216427 non-null float64
PMD_MEDICINALNAME             216427 non-null object
PMD_COMPANYNAME               216427 non-null object
PMD_COMPANYID                 216427 non-null object
PMD_MEDICINALINFORMATIONID    216427 non-null object
PMD_MEDICINALID               216427 non-null object
PMD_ID                        0 non-null object
PMD_REMARK1                   216427 non-null object
PMD_REMARK2                   0 non-null object
PMD_REMARK3                   0 non-null object
PMD_REMARK4                   0 non-null object
PMD_REMARK5                   0 non-null object
CREATE_DATE   

In [11]:
uniquekeys=['PMD_YEAR', 'PMD_QUARTER', 'PMD_CITY', 'PMD_MEDICINALINFORMATIONID', 'PMD_COMPANYID',
             'PMD_MEDICINALSALENUMBER']

In [16]:
crcmax = data.groupby(uniquekeys).ETL_CRC.transform(max)

In [24]:
crcmax = data.groupby(uniquekeys).agg('count') #ETL_CRC.transform(max)

In [34]:
data.to_csv("drugsale_test_data.csv", index=False, encoding='utf-8')

### groupby 

### count()会作用在所有不在uniquekeys给定的列上，生成一个由uniquekeys做多级索引的DateFrame。比如df中的列为[a,b,c,d,e], uniquekeys=[a,b,c], 那么count()会同时作用在[d,e]列上， 而[a,b,c]成为多级索引。info()可以看到相关信息。通常没有必要在[d,e]列上各做一次count，当df中的列数特别多时，就更不推荐这么用了。

In [50]:
# data.groupby(uniquekeys).count().info()#.reset_index()

### reset_index()可以多级索引转换为普通的列。

In [51]:
# data.groupby(uniquekeys).count().reset_index()

### size()的结果中只会保留uniquekeys中存在的列, 并新增一个名叫'counts'的计数列。 也是呈目录树状的级联结构，但它不是一个DateFrame, 查看类型会发现是Series。可以用reset_index转换成DateFrame。

In [46]:
type(data.groupby(uniquekeys).size())

pandas.core.series.Series

### reset_index()直接作用在size()后面，再赋予新增的计数列列名'counts'可以得到一个看上去干净清爽的DateFrame, 同时满足获取各个group的计数的需求。

In [55]:
# data.groupby(uniquekeys).size().reset_index(name='counts')

### size()方法后直接用to_frame()方法也可以得到DateFrame, 它只有唯一一列'counts', info()显示uniquekeys为这个DataFrame的多级索引。reset_index()方法仍然可以将这些索引转换成列，而自动加上默认的自增索引。

In [45]:
data.groupby(uniquekeys).size().to_frame(name='counts').info()#.reset_index()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 73054 entries, (2013, 1, 上海, D0000100100LCD0131T001, M1359, 120.0) to (2013, 1, 长沙, D0113210100BAA0163T030, M4862, 139800.0)
Data columns (total 1 columns):
counts    73054 non-null int64
dtypes: int64(1)
memory usage: 1.5+ MB


In [56]:
# data.groupby(uniquekeys).size().to_frame(name='counts').reset_index()

In [19]:
df = data.loc[data.ETL_CRC==crcmax]

In [22]:

df = df.reset_index()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108966 entries, 0 to 108965
Data columns (total 38 columns):
index                         108966 non-null int64
ID                            108966 non-null int64
PMD_YEAR                      108966 non-null int64
PMD_QUARTER                   108966 non-null int64
PMD_CITY                      108966 non-null object
PMD_MEDICINALSALEMONEY        108966 non-null float64
PMD_MEDICINALSALENUMBER       108966 non-null float64
PMD_MEDICINALNAME             108966 non-null object
PMD_COMPANYNAME               108966 non-null object
PMD_COMPANYID                 108966 non-null object
PMD_MEDICINALINFORMATIONID    108966 non-null object
PMD_MEDICINALID               108966 non-null object
PMD_ID                        0 non-null object
PMD_REMARK1                   108966 non-null object
PMD_REMARK2                   0 non-null object
PMD_REMARK3                   0 non-null object
PMD_REMARK4                   0 non-null object
PMD_REMARK

In [99]:
df_drugssales_merge.columns=[ u'PMD_CITY',u'PMD_COMPANYID',u'PMD_COMPANYNAME',u'PMD_MEDICINALID',u'PMD_MEDICINALINFORMATIONID',u'PMD_MEDICINALSALEMONEY',u'PMD_MEDICINALSALENUMBER',u'PMD_YEAR', u'PMD_QUARTER',u'PMD_MEDICINALNAME',u'PMD_USEMETHOD',u'PMD_REMARK1',u'PMD_DOSFORM',u'PMD_SPECIFICIAL',u'PMD_CLASS']
df_drugssales_merge.head()

,PMD_CITY,PMD_COMPANYID,PMD_COMPANYNAME,PMD_MEDICINALID,PMD_MEDICINALINFORMATIONID,PMD_MEDICINALSALEMONEY,PMD_MEDICINALSALENUMBER,PMD_YEAR,PMD_QUARTER,PMD_MEDICINALNAME,PMD_USEMETHOD,PMD_REMARK1,PMD_DOSFORM,PMD_SPECIFICIAL,PMD_CLASS
0,北京市,M1316,贵州信邦制药,ZA0600-D003111,D0031110200AAA0925T082,928428.3566,25788.0,2013,1,护肝宁,内服,100,片剂,350mg,2
1,北京市,M3809,四川金辉药业,ZF0300-D003587,D0035870500AAA1238T045,67717.9205,3214.0,2013,1,金刚藤,内服,48,片剂,520mg,2
2,北京市,M2425,江西济民可信药业,ZF0300-D003910,D0039100400AAA1254T056,471059.4568,20910.0,2013,1,抗宫炎,内服,60,片剂,550mg:500mg,2
3,北京市,M1847,呼伦贝尔松鹿制药,ZE0300-D007564,D0075640100AAA1156T056,48309.6656,2005.0,2013,1,小儿胃宝,内服,60,片剂,500mg,2
4,北京市,M3569,上海医药青岛国风药业,ZC0300-D007902,D0079020100AAA1332T035,298423.1510,9392.0,2013,1,养心氏,内服,36,片剂,600mg,2


In [62]:
# df_drugssales_merge.head()

,PMD_CITY,PMD_COMPANYID,PMD_COMPANYNAME,PMD_MEDICINALID,PMD_MEDICINALINFORMATIONID,PMD_MEDICINALSALEMONEY,PMD_MEDICINALSALENUMBER,PMD_YEAR,PMD_QUARTER,PMD_REMARK1,PMD_DOSFORM,PMD_SPECIFICIAL,PMD_CLASS
0,北京市,M1316,贵州信邦制药,ZA0600-D003111,D0031110200AAA0925T082,928428.3566,25788.0,2013-03-31,1,100,片剂,350mg,2
1,北京市,M3809,四川金辉药业,ZF0300-D003587,D0035870500AAA1238T045,67717.9205,3214.0,2013-03-31,1,48,片剂,520mg,2
2,北京市,M2425,江西济民可信药业,ZF0300-D003910,D0039100400AAA1254T056,471059.4568,20910.0,2013-03-31,1,60,片剂,550mg:500mg,2
3,北京市,M1847,呼伦贝尔松鹿制药,ZE0300-D007564,D0075640100AAA1156T056,48309.6656,2005.0,2013-03-31,1,60,片剂,500mg,2
4,北京市,M3569,上海医药青岛国风药业,ZC0300-D007902,D0079020100AAA1332T035,298423.1510,9392.0,2013-03-31,1,36,片剂,600mg,2
